# Técnicas de Limpeza e Tratamento de Valores Ausentes para Análise de Dados.

## Pacotes Python usados no projeto

In [1]:
!pip install -q -U watermark

In [2]:
# imports
import math
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

In [3]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



## Carregando os dados
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [4]:
# Criando uma lista para identificar possíveis representações para valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

In [5]:
# Carregando o dataset com valores ausentes
# O parâmetro na_values identifica os argumentos de entrada como valores NaN. A ultilização
# deste parâmetro é valida quando se sabe de ante
dataset_dsa = pd.read_csv("dataset.csv", na_values= lista_labels_valores_ausentes)

In [6]:
# Shape
dataset_dsa.shape

(150001, 55)

In [7]:
# Alteração de parâmetro do Pandas "display.max_columns" para almentar o número de colunas exibidas ao imprimir o dataframe.
pd.set_option('display.max_columns', 100)
# Caso o df tenha mais de 100 colunaas, o que passar será trucado.

In [8]:
# Carregando o dicionário de dados
dicionario = pd.read_excel('dicionario.xlsx')

In [9]:
# Shape
dicionario.shape

(56, 2)

In [10]:
# Definindo um valor grande para a largura da coluna.
pd.set_option('display.max_colwidth', 100)

In [11]:
# Amostra de dados
# dicionario.head()

## Análise Exploratória

In [12]:
# info
dataset_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [13]:
# Estatísticas descritivas
# dataset_dsa.describe()

Não faz sentido calcular estatísticas descritivas para variáveis Baread id, IMSI, MSISDN/Number e IMEI Embora essas colunas sejam
do tipo numérico, seus valores servem como identificadores. Mas o método describe() calcula as estatísticas de todas as colunas numéricas. Essas estatísticas estão sendo calculadas antes que os dados sejam limpos. Portanto, pode haver mudanças depois que os valores ausentes e outliers são tratados.

In [10]:
# Shape
dataset_dsa.shape

(150001, 55)

In [11]:
dicionario.shape

(56, 2)

Existem 150.001 linhas e 55 colunas no dataframe. No entando, temos 56 colunas com seus nomes e descrições no dicionário.
Isso significa que há uma coluna descrita, mas não incluída no dataframe.

In [14]:
# Concatenar os datasets para comparação de variáveis (lado a lado)
df_compara_colunas = pd.concat([pd.Series(dataset_dsa.columns.tolist()), dicionario['Fields']], axis=1)

In [10]:
df_compara_colunas.columns

Index([0, 'Fields'], dtype='object')

In [15]:
# Renomeia as colunas do df_compara_colunas
df_compara_colunas.rename(columns = {0:'Coluna do Dataset', 'Fields': 'Coluna do Dicionário'}, inplace = True)

In [12]:
# df_compara_colunas

"Dur.(Ms)" é ignorado no dataset como visto no indice 1 em df_compara_colunas. É ai que a ordem das colunas começa a mudar.
Mas o mesmo nome da coluna "Dur.(Ms)" aparece no dataser no indice 5, enquanto o arquivo de dicionário nos diz que é "Dur.(S)"
no indice 6. Como as medidas de ambas as colunas diferem conforme mostrado em seus nomes, nós precisamos verificar qual está
certo. Para investigar isso, usaremos a coluna "Dur.(Ms).1" que se encontra nos indices 28 e 29 no dataset e no arquivo de dicionário,
respectivamente.

In [13]:
# Filtro de colunas no df dataset_dsa para verificar se os valores das variáveis são identicos mas com
# o nome de coluna diferente. (duplicação de variável).
dataset_dsa[['Dur. (ms)', 'Dur. (ms).1']]

,Dur. (ms),Dur. (ms).1
0,1823652.0,1.823653e+09
1,1365104.0,1.365104e+09
2,1361762.0,1.361763e+09
3,1321509.0,1.321510e+09
4,1089009.0,1.089009e+09
...,...,...
149996,81230.0,8.123076e+07
149997,97970.0,9.797070e+07
149998,98249.0,9.824953e+07
149999,97910.0,9.791063e+07


Parece que a coluna "Dur. (Ms)" é medida em segundos. Portanto, vamos renomeá-la apropriadamente. Vamos também renomear
algumas colunas para que fiquem claras como sua descrição e sigam o estilo de nomenclatura de outras colunas.

In [16]:
# Renomear as colunas do df dataset_dsa (parâmetro inplace = True para salvar no próprio df).
dataset_dsa.rename(columns= {'Dur. (ms)': 'Dur (s)',
                             'Dur. (ma).1': 'Dur (ms)',
                             'Start ms' : 'Start Offset (ms)',
                             'End ms': 'End Offset (ms)'},
                   inplace = True)

In [17]:
# Lista de colunas do dataset
dataset_dsa.shape

(150001, 55)

## Etapa 1 - Tratamento de Valores Ausentes

- 1- Identificando Valores Ausentes
- 2- Drop de Colunas
- 3- Imputação com Preenchimento Reverso
- 4- Imputação com Preenchimento Progressivo
- 5- Imputação de Variáceis Categóricas
- 6- Drop de Linhas

## 1.1 Identificando Valores Ausentes

In [16]:
print(dataset_dsa.shape)
np.product(dataset_dsa.shape)

(150001, 55)


8250055

In [18]:
# Função que calcula o percentual de valores ausentes no dataset (Geral)
def calc_percent_valores_ausentes(df):
    #calcula o total de celulas do df
    total_cells = np.product(df.shape)

    # conta o número de valores ausentes por coluna
    missing_count = df.isnull().sum()

    # calcula o tatal de valores ausentes
    totalMissing = missing_count.sum()

    # Calcula o percentual de valores ausentes
    print("O dataset tem", round(((totalMissing / total_cells) * 100), 2), "%", "de valores ausentes.")

In [12]:
# Chamando a função para descobrir o percentual de valores ausentes no df
calc_percent_valores_ausentes(dataset_dsa)

O dataset tem 12.72 % de valores ausentes.


In [19]:
# função para calcular valores ausentes por coluna.
def calc_perc_val_aus_cols(df):
    # Total de valores ausentes
    mis_val = df.isnull().sum()
    # Percentual de valores ausentes
    mis_val_percent = 100 * mis_val / len(df)
    # Tipo de dados das colunas com valores ausentes
    mis_val_dtypes = df.dtypes
    # Criando uma tabela com os resultados
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtypes], axis=1)

    # Renomear as colunas
    mis_val_table_ren_columns = mis_val_table.rename(
            columns = {0: 'Valores Ausentes', 1: '% de Valores Ausentes', 2: 'Dtypes'})
    # Classifica a tabela por porcentagem de valores ausentes de forma decresvcente e remove colunas sem valores ausentes
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)
    
    # Print
    print("O dataset tem " + str(df.shape[1]) + " colunas.\n"
         "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")
    if mis_val_table_ren_columns.shape[0] == 0:
        return

    # Retorna o dataframe com informações ausentes
    return mis_val_table_ren_columns

In [21]:
# Criar tabelas com valores ausentes
df_missing = calc_perc_val_aus_cols(dataset_dsa)

O dataset tem 55 colunas.
Encontrado: 41 colunas que têm valores ausentes.


Normalmente, colunas com mais de 50% de valores ausentes devem ser removidas. Entre 30 e 50% é opcional.

Mas a última é sempre do analista de dados! Lembrar-se sempre se justificar a decisão.

Neste projeto vamos remover colunas cujos valores ausentes representam mais de 30% da variável, pois temos um número
muito grande de colunas com valores ausentes e, portanto, muito trabalho. Vamos tratar as variáveis com percentual baixo e deletar
aquelas que tiverem percentual alto de valores ausentes.

## 1.2 Drop de Colunas

In [22]:
# Criando um df filtrando as colunas candidata ao a serem removidas.
colunas_para_remover = df_missing[df_missing['% de Valores Ausentes'] >= 30.00].index.tolist()

In [17]:
# Colunas que serão removidas
colunas_para_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'TCP UL Retrans. Vol (Bytes)',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'TCP DL Retrans. Vol (Bytes)',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

Mesmo que as variáveis "TCP" tenham muitos valores ausentes, em vez removê-las, o instrutor irá ensinar como imputar valores a
essas variáveis, uma vez que elas podem ser necessárias para a análise posterior.

In [30]:
# Colunas que serão removidas
colunas_para_remover = [col for col in colunas_para_remover if col not in ['TCP UL Retrans. Vol (Bytes)', 'TCP DL Retrans. Vol (Bytes)']]

In [31]:
# Colunas que serão removidas
colunas_para_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [32]:
# Drop das colunas e criação de outro df
dataset_limpo = dataset_dsa.drop(colunas_para_remover, axis=1)

In [33]:
# Shape
dataset_limpo.shape

(150001, 47)

Verifica novamente o status dos valores ausentes no dataframe modificado.

In [34]:
calc_percent_valores_ausentes(dataset_limpo)

O dataset tem 3.85 % de valores ausentes.


In [59]:
# calc_perc_val_aus_cols(dataset_limpo)

## 1.3. Imputação com Preechimento Reverso

Uma vez que as porcentagens de valores ausentes 'TCP UL Retrans. Vol(Bytes)' e 'TCP DL Retrans. Vol (Bytes)'são muito altos,
iremos aplicar imputação nos valoores com o método de preenchimento reverso.
Nesse caso, usar um único valor como média ou mediana não é aconselhavel, pois pode alterar nossos dados de uma forma indesejavel, tornando a maioria dos valores igual a um único valor.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

In [36]:
# Imputação de valores ausentes usando o Backward fill
# method = 'bfill': Bfill ou backward-fill propaga o primeiro valor não nulo observado para trás até que
# outro valor não nulo seja encontrado.
def func_fix_missing_bfill(df, col):
    
    count = df[col].isna().sum()

    df[col] = df[col].fillna(method = 'bfill')

    print(f"{count} valores ausentes na coluna {col} foram substituídos usando o método de preenchimento reverso.")

In [37]:
# Imputação com preechimento Reverso na variável 'TCP UL Retrans. Vol (Bytes)'
func_fix_missing_bfill(dataset_limpo, 'TCP UL Retrans. Vol (Bytes)')

96649 valores ausentes na coluna TCP UL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


In [38]:
# Imputação com preechimento Reverso na variável 'TCP DL Retrans. Vol (Bytes)'
func_fix_missing_bfill(dataset_limpo, 'TCP DL Retrans. Vol (Bytes)')

88146 valores ausentes na coluna TCP DL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


## 1.4. Imputação com Preenchimento Progressivo

In [58]:
# calc_perc_val_aus_cols(dataset_limpo)

Avg RTT DL(ms) e Avg RTT UL(ms) têm as próximas as próximas porcentagens de valores ausentes com cerca de 18,5% cada. Vamos verificar se as variáveis estão enviesadas (não seguem um distribuição normal) usando o método skew(), que retorna o coeficiênte de assimetria.

In [40]:
dataset_limpo['Avg RTT DL (ms)'].skew(skipna = True)

62.90782807995961

In [41]:
dataset_limpo['Avg RTT UL (ms)'].skew(skipna = True)

28.45741458546382

 - Se a assimetria estiver entre -0,5 e 0,5, os dados são bastante simétricos
 - Se a assimetria estiver entre -1 e -0,5 ou entre 0,5 e 1, os dados estão moderadamente inclinados
 - Se a assimetria for menor que -1 ou maior que 1, os dados estão altamente enviesados

Visto que ambas as colunas Avg RTT DL (ms) e Avg RTT UL (ms) são fortemente enviesadas positivamente é aconcelhável não imputá-las com sua média. Portanto usamos o preenchimento progressivo.

In [42]:
# Imputação de valores ausentes usando forward fill (preenchimento progressivo)
# method = 'ffill' : Ffill ou forward-fill propaga o último valor não nulo abservado para frente até que
# outro valor não nulo seja encontrado
def func_fix_missing_ffill(df, col):

    count = df[col].isna().sum()

    df[col] = df[col].fillna(method = 'ffill')

    print(f"{count} valores ausentes na coluna {col} foram substituídos usando o método de preenchimento progressivo.")

In [43]:
# Imputação com preenchimento progressivo
func_fix_missing_ffill(dataset_limpo,'Avg RTT DL (ms)') 

27829 valores ausentes na coluna Avg RTT DL (ms) foram substituídos usando o método de preenchimento progressivo.


In [44]:
# Imputação com preenchimento progressivo
func_fix_missing_ffill(dataset_limpo,'Avg RTT UL (ms)') 

27812 valores ausentes na coluna Avg RTT UL (ms) foram substituídos usando o método de preenchimento progressivo.


In [45]:
calc_percent_valores_ausentes(dataset_limpo)

O dataset tem 0.44 % de valores ausentes.


In [57]:
# calc_perc_val_aus_cols(dataset_limpo)

In [52]:
dataset_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Bearer Id                       149010 non-null  float64
 1   Start                           150000 non-null  object 
 2   Start Offset (ms)               150000 non-null  float64
 3   End                             150000 non-null  object 
 4   End Offset (ms)                 150000 non-null  float64
 5   Dur (s)                         150000 non-null  float64
 6   IMSI                            149431 non-null  float64
 7   MSISDN/Number                   148935 non-null  float64
 8   IMEI                            149429 non-null  float64
 9   Last Location Name              148848 non-null  object 
 10  Avg RTT DL (ms)                 150001 non-null  float64
 11  Avg RTT UL (ms)                 150001 non-null  float64
 12  Avg Bearer TP DL

[Visto que "Handset Type" e "Handset Manufacturer" são colunas categóricas, é melhor imputá-las com  o valor "unknown"
para que não enviesemos os dados.

In [53]:
# Preencher valor NA
def func_fix_missing_value(df, col, value):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(value)
    if type(value) == 'str':
        print(f"{count} valores ausentes na coluna {col} foram substituídos por '{value}'.")
    else:
        print(f"{count} valores ausentes na coluna {col} foram substituídos por {value}.")

In [54]:
# Imputação de variáveis categóricas
func_fix_missing_value(dataset_limpo, 'Handset Type', 'unknown')

0 valores ausentes na coluna Handset Type foram substituídos por unknown.


In [55]:
func_fix_missing_value(dataset_limpo, 'Handset Manufacturer', 'unknown')

0 valores ausentes na coluna Handset Manufacturer foram substituídos por unknown.


In [56]:
calc_percent_valores_ausentes(dataset_limpo)

O dataset tem 0.17 % de valores ausentes.
